## U shape PHP dynamics with multiple liquid slugs - with heat transfer

## wall heat equation

## $\frac{\partial \theta }{\partial \mathcal{T}} = \nabla^2 \theta + W_{evaporator} + H_{condenser}(\theta_{condenser} - \theta_{wall}) + H_{fluid}(\theta_{fluid} - \theta_{wall})$

## liquid heat equation

## $\frac{\partial \theta }{\partial \mathcal{T}} = \nabla^2 \theta + H_{fluid}(\theta_{wall} - \theta_{fluid})$

## dynamics equations
We solve a set of governing equations:

1. Momentum equation:
 ## $\frac{\mathrm{d}^{2} X_{p}}{\mathrm{d} \tau^{2}}+32 \frac{\mathrm{d} X_{p}}{\mathrm{d} \tau}+\omega_{0}^{2} X_{p}=\wp\left(P_{1}-P_{2}\right)$
 
2. Perfect gas law combined with first law of thermodynamics:

 ## $M_{i}=P_{i}^{1 / \gamma}L_{i}$
 
 ## $\theta_{i}=P_{i}^{(\gamma-1) / \gamma}$
 


3. The phase change relations between the mass of vapor and the heat transfer from the wall
 ## $\frac{\mathrm{d} M_{i}}{\mathrm{d} \mathcal{T}}=\int_{X_{start}}^{X_{end}} H_{vapor} (\theta_{wall} - \theta_{vapor}) dX$

 


## nucleate boiling

$nucleatenum = 2$ is the number of nucleation sites. It is distributed randomly for every start time.

$\Delta t = 0.1$ is the time interval for nucleations, for now it is uniform for all nucleation sites, can be modified later on.

$Δθthreshold = 0.01$ is the super heat threshold.

So it means that at time $\Delta t$,  $2\Delta t$,  $3\Delta t$ ... and so on. It will check if the nucleation sites are in the liquid slug region and if the superheat is fulfilled. if so, it will reslease one or several $L=0.02, P=1$ new vapor bubbles

In [1]:
using Revise
using LinearAlgebra
using DifferentialEquations
using Plots
pyplot()

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise C:\Users\liyux\.julia\packages\Revise\1boD5\src\packagedef.jl:1369


Plots.PyPlotBackend()

In [81]:
cd(dirname(pwd()))
cd("src") 

includet("OneDOHP.jl")
using ..OneDOHP

## Parameters of the PHP

In [82]:
    ω = sqrt(1.68e003);
    ℘L = 6.2e004;
    k = 115e-003

    θinitial=1.0
    θc = 1.0; # useless

    Hwc = 0.0; #not useful later on. H between wall and condenser, 
    Hₗ   =  1600.0; # liquid wall heat transfer coeffcient
#     Hₗ   =  2.57e004; # liquid wall heat transfer coeffcient
    Hδ  = 7.34e002;


In [83]:
    
    d = 6.56e-3 # dimensionless diameter
    L = 35.0  # total length of the pipe when streched to a 1D pipe
    L2D = 1.0 # the actual length of the bended pipe in the real world
    angle = 0*pi/2 # inclination angle 
    closedornot = true

true

In [84]:
    nucleatenum = 10
    Xstations = sort(rand(nucleatenum).*L);
#     boilingΔt = 0.05

## Pipe definition

In [85]:
tube = Tube(d,L,L2D,angle,closedornot);

## Evaporator and condenser definition

In [86]:
Xe = map(tuple, [1.0], [3.0])
We = [0.0]
evap = Evaporator(Xe,We);

In [87]:
Xc = map(tuple, [0.0,3.0], [1.0,4.0])
cond = Condenser(θc,Xc,Hwc);

## Vapor plug and liquid slug initialization

In [88]:
numofX=16
Xps = sort(rand(2*numofX).*L);
Xp1s = Xps[1:2:end];
Xp2s = Xps[2:2:end];

dXdt0 = map(tuple,0*Xp1s,0*Xp2s)
X0 = map(tuple,Xp1s,Xp2s)


16-element Array{Tuple{Float64,Float64},1}:
 (0.41656075129194226, 0.6077565580304656)
 (0.702898027024631, 2.2258842669376087)
 (2.993424401275181, 4.412829890825279)
 (6.6745096462266424, 7.026143041234149)
 (7.476459242620772, 8.093372945435732)
 (8.2403333135614, 8.878562316676419)
 (9.143620951835226, 9.406964918696023)
 (10.23172868872538, 13.228630910466123)
 (13.390928778689151, 14.421920147001183)
 (15.33943330303503, 18.219705777270548)
 (19.153622291157326, 23.550203042739817)
 (23.956013124045985, 25.267438088802106)
 (26.65253055124079, 26.90508188810497)
 (28.063265524511184, 29.030327000359495)
 (32.22460637007288, 32.44999546885903)
 (34.139778655318494, 34.43174387188878)

In [89]:
N=6000
# X0 = [(L/4, L*3/4)]
# dXdt0 = [(0.0,0.0)]
ρ = 214.6
Xarrays,θarrays = constructXarrays(X0,N,θinitial,L)
liquids=Liquid(Hₗ,ρ,ω,℘L,X0,dXdt0,Xarrays,θarrays);

In [90]:
N=500
X0 = [(1.0, 3.0)]
dXdt0 = [(0.0,0.0)]
ρ = 1000
Xarrays,θarrays = constructXarrays(X0,N,θinitial,L)
liquids=Liquid(Hₗ,ρ,ω,℘L,X0,dXdt0,Xarrays,θarrays);

In [91]:
# P = [1.0,1.0,1.0,1.0,1.0]; # closed end

γ = 1.4;
Hδ = 7.34e2
P = 0*Xp1s .+ 1.0;
δ = 0*Xp1s .+ 2.86e-02;
vapors=Vapor(γ,Hδ,P,δ);

In [92]:
α = 0 # nondimensional thermal diffusivity = (αₐ d^2) / (νₗ height^2)
Xwallarray,θwallarray = constructXarrays(L,N,θinitial)

wall = Wall(α,Δt,Xstations,Xwallarray,θwallarray);

In [93]:
walltoliquid, liquidtowall = constructmapping(Xarrays,Xwallarray,closedornot,L)
mapping = Mapping(walltoliquid,liquidtowall);

## Construct the system

In [94]:
sys0 = PHPSystem(tube,evap,cond,liquids,vapors,wall,mapping);

In [95]:
typeof(sys0)

PHPSystem

## Construct U for wall heat function

## construct state vector U

In [101]:
Lvaporplug = XptoLvaporplug(X0,sys0.tube.L,sys0.tube.closedornot)
M = P.^(1/γ).* Lvaporplug


u=[XMδtovec(X0,dXdt0,M,δ); wallθtovec(sys0.wall.θarray); liquidθtovec(sys0.liquid.θarrays)];

the lengthes of X and dXdt and M do not match!


In [102]:
# plot([sys0.wall.Xarray],[sys0.wall.θarray], title="initial temperature condition", labels = "θ", xlabel="X", ylabel="θ")

## Solve the constructed ODE for U

In [103]:
cb_boiling =  DiscreteCallback(boiling_condition,boiling_affect!)
cb_merging =  DiscreteCallback(merging_condition,merging_affect!)

cb = CallbackSet(cb_boiling,cb_merging)

CallbackSet{Tuple{},Tuple{DiscreteCallback{typeof(boiling_condition),typeof(boiling_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)},DiscreteCallback{typeof(merging_condition),typeof(merging_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}}}((), (DiscreteCallback{typeof(boiling_condition),typeof(boiling_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}(boiling_condition, boiling_affect!, DiffEqBase.INITIALIZE_DEFAULT, DiffEqBase.FINALIZE_DEFAULT, Bool[1, 1]), DiscreteCallback{typeof(merging_condition),typeof(merging_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}(merging_condition, merging_affect!, DiffEqBase.INITIALIZE_DEFAULT, DiffEqBase.FINALIZE_DEFAULT, Bool[1, 1])))

In [104]:
tstep=1e-4

p = sys0
tspan = (0.0,0.1)
prob = ODEProblem(ODE_innertube, u, tspan, p)

integrator = init(prob, RK4(), dt=tstep, callback=cb);


solve!(integrator)



LoadError: [91mStackOverflowError:[39m

In [77]:
p = deepcopy(getcurrentsys(integrator.sol[end],sys0));
p.liquid.Xp

LoadError: [91mUndefVarError: integrator not defined[39m

In [78]:
sol = integrator.sol
sysfinal = getcurrentsys.(sol.u,[sys0]);

LoadError: [91mUndefVarError: integrator not defined[39m

In [79]:
# closed loop plot
@gif for i=1:500:length(sysfinal)

    Xpvapor = getXpvapor(sysfinal[i].liquid.Xp,sysfinal[i].tube.L,sysfinal[i].tube.closedornot)
    θvapor  = (sysfinal[i].vapor.P).^((γ-1)/γ)
    Xp = sysfinal[i].liquid.Xp
    
    p = plot([sysfinal[i].wall.Xarray],[sysfinal[i].wall.θarray], linewidth=3, color = [:black], labels = "θwall", xlabel="X", ylabel="θ", title=("dimensionless temperatures"),xlim=(0,4), ylim=(0.8,1.2))
    
    
    j=1
    while j <= length(Xp)
        if Xp[j][end] >= Xp[j][1]
            plot!(p, [sysfinal[i].liquid.Xarrays[j]],[sysfinal[i].liquid.θarrays[j]], linewidth=2, labels = string("θliquid ",j))
            else 
            # find the index at the end
            index = findfirst(x->x <= sysfinal[i].liquid.Xarrays[j][end], sysfinal[i].liquid.Xarrays[j])
            
            plot!(p, [sysfinal[i].liquid.Xarrays[j][1:index-1]],[sysfinal[i].liquid.θarrays[j][1:index-1]], linewidth=2, labels = string("θliquid ",j))
            plot!(p, [sysfinal[i].liquid.Xarrays[j][index:end]],[sysfinal[i].liquid.θarrays[j][index:end]], linewidth=2, labels = string("θliquid ",j))

        end
        
        j += 1
    end

        j=1
    while j <= length(Xpvapor)
        if Xpvapor[j][end] >= Xpvapor[j][1]
            plot!(p,[Xpvapor[j][1],Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
            else
            
            plot!(p,[0.0,Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
            plot!(p,[Xpvapor[j][1],L],[θvapor[j]], labels = string("θvapor ",j))

        end
        
        j += 1
    end
    
#     j = 1
#     plot!(p,[0.0,Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
#     plot!(p,[Xpvapor[j][1],L],[θvapor[j]], labels = string("θvapor ",j))
    
#     j = 2
#     while j  <= length(θvapor) 
    
#         plot!(p,[Xpvapor[j][1],Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
        
#         j += 1
#     end

    vline!( p, [collect(0:2:4)], color=[:blue,:red], width=196, alpha=[0.3,0.3,0.3],labels = "")
    
    end 

LoadError: [91mUndefVarError: sysfinal not defined[39m

In [24]:
constant = 2.5e6/461.5/300

18.057060310581438

In [25]:
function onePtooneT(P,constant)
    1/(1-(log(P)/constant))
end

onePtooneT (generic function with 1 method)

In [26]:
onePtooneT(1.0,constant)

1.0

In [27]:
XX = constructXarrays(L::Float64,N,θinitial)

XX[1][1]

0.0

In [28]:
plot(sys_to_heatflux(sysfinal[end]))

LoadError: [91mUndefVarError: sysfinal not defined[39m

In [29]:
@time ODE_innertube(u,sys0,0.0)

LoadError: [91mUndefVarError: u not defined[39m

In [30]:
@time ODE_innertube(u,sys0,0.0);

LoadError: [91mUndefVarError: u not defined[39m

In [31]:
@time index_dynamics_end = findfirst(x->abs(x+1e10) <= 10^(-1), u)

LoadError: [91mUndefVarError: u not defined[39m

In [32]:
@time newsys = getcurrentsys(u,sys0)

LoadError: [91mUndefVarError: u not defined[39m

In [33]:
cd(dirname(pwd()))
cd("src") 

includet("OneDOHP.jl")
using ..OneDOHP